In [ ]:
from api.resources.blender import Blender



In [ ]:
import pandas as pd
import inspect
import bpy
# create a new geometry node tree
tree = bpy.data.node_groups.new(type='GeometryNodeTree', name='GeometryNodes')


# Create a new mesh data block
gridmesh = bpy.data.meshes.new('GridMesh')

if "Cube" in bpy.data.meshes:
    mesh = bpy.data.meshes["Cube"]
    print("removing mesh", mesh)
    bpy.data.meshes.remove(mesh)

# Create a new object that uses the mesh data block
obj = bpy.data.objects.new('Grid', gridmesh)
print(list(bpy.data.objects))

# Add the object to the scene collection
bpy.context.scene.collection.objects.link(obj)


print("\n--")
print("Estos son los objetos de tu escena")

    
# Leer el archivo CSV
print("\n--")
print("Este es tu Csv")
df = pd.read_csv('modelo.csv')
print(df)



separacion_muestras = 3
# Crear una visualización 3D de los datos
for i, row in df.iterrows():
    x, y, z, value = row['numeromuestra'], row['intensidad'], row['aceleracion'], row['altura']
    # Create a UV sphere

    mat = bpy.data.materials.new(name="SphereMaterial")
    mat.diffuse_color = (2**y, 0, 0, 0.5)  # Red color
    
    # Get the active object (which is the sphere we just created)
    bpy.ops.mesh.primitive_uv_sphere_add(location=(x*(separacion_muestras), y, 1), segments = z, ring_count = y, radius = value, calc_uvs = True) 
    sphere = bpy.context.active_object
    # Add a material to the sphere
    
    
    # Set the color of the material
    sphere.data.materials.append(mat)

# Exportar el modelo a SVG
output_file = "file.gltf"

# Set export options
export_settings = {
    "filepath": output_file,
    "check_existing": False,
    "export_colors": True,
    "export_animations": False,

}

# Export the active object
bpy.ops.export_scene.gltf(**export_settings)


In [6]:
import bpy
import bmesh

# Create a new mesh
mesh = bpy.data.meshes.new("Cube")

# Create a new object using the mesh
object = bpy.data.objects.new("Cube", mesh)

# Create a new bmesh and add vertices and edges
bm = bmesh.new()
verts = [bm.verts.new((x,y,z)) for x in (-1,1) for y in (-1,1) for z in (-1,1)]
print(len(verts))
edges = [bm.edges.new((verts[i], verts[j])) for i in range(len(verts)) for j in range(i+1,len(verts)) if sum((a-b)**2 for a,b in zip(verts[i].co,verts[j].co)) == 1]


# Create faces connecting each set of 3 vertices that form a corner of the cube

faces = [bm.faces.new((verts[i],verts[j],verts[k])) for i in range(-2,len(verts),2) for j in range(i-1,i+1) for k in range(j-1.5,i+1.5)]

# Assign the bmesh to the mesh
bm.to_mesh(mesh)

# Assign a new material to the cube
material = bpy.data.materials.new(name="Red")
material.diffuse_color = (1,0,0,1)
object.data.materials.append(material)

# Link the object to the scene and select it
scene = bpy.context.scene
scene.collection.objects.link(object)
object.select_set(True)
# Exportar el modelo a SVG
output_file = "file.gltf"

# Set export options
export_settings = {
    "filepath": output_file,
    "check_existing": False,
    "export_colors": True,
    "export_animations": False,

}

# Export the active object
bpy.ops.export_scene.gltf(**export_settings)



8


TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
import bpy
def create_material():
        

    # create a new material
    mat = bpy.data.materials.new(name="MySphereMaterial")

    # make the material red using nodes
    mat.use_nodes = True
    nodes = mat.node_tree.nodes
    for node in nodes:
        nodes.remove(node)

    diffuse_node = nodes.new(type='ShaderNodeBsdfDiffuse')
    diffuse_node.inputs[0].default_value = (1, 0, 0, 1)
    transparent_node = nodes.new(type='ShaderNodeBsdfTransparent')
    transparent_node.inputs[0].default_value = (1, 1, 1, 1)
    transparent_node.inputs[1].default_value = 0.5
    mix_node = nodes.new(type='ShaderNodeMixShader')
    mix_node.inputs[0].default_value = 0.5
    output_node = nodes.new(type='ShaderNodeOutputMaterial')
    links = mat.node_tree.links
    links.new(diffuse_node.outputs[0], mix_node.inputs[1])
    links.new(transparent_node.outputs[0], mix_node.inputs[2])
    links.new(mix_node.outputs[0], output_node.inputs[0])
    return mat



In [ ]:
import bpy


# Any Python object can act as the subscription's owner.


owner = bpy.data.objects["Cube"]
    
subscribe_to = (bpy.types.Object, "location")


owner.location.x +=1 
bpy.data.objects["Light"].location = 2,2,2

def msgbus_callback(*args):
    print(args)
    print("hi")
    # This will print:
    # Something changed! (1, 2, 3)
    print("Something changed!", args)
    return 


bpy.msgbus.subscribe_rna(
    key=subscribe_to,
    owner=owner,
    args=(1,2,2),
    notify=msgbus_callback(0,1,2),
)


print(owner.location)